# 🧩 Transducible Functions Tutorial

**Transducible functions** are Python functions that can be *lifted* into **Agentic Structures (AGs)** — 
structured, traceable computational units used in the `agentics` framework.

They extend ordinary Python functions with GenAI built in capabilities by:
- Capturing structured **input/output models**
- Supporting **async execution**
- Integrating with LLMs using **Agentics Transductional Engine**
- Supporting **Parallel mapping (amap)** operations

In this tutorial, you’ll learn how to:
1. Define and decorate transducible functions  
2. Execute them both singly and in parallel
3. Use the framework to enable Map Reduce


### Defining Transducible Functions

Agentics allows you to wrap any Python function and turn it into a transducible function, giving it native LLM-powered transformation capabilities.


#### Defining Source and Target Types

In [1]:
from typing import Optional
from pydantic import BaseModel, Field
from agentics.core.transducible_functions import transducible, Transduce

class GenericInput(BaseModel):
    content: Optional[str] = None

class Email(BaseModel):
    to: Optional[str] = None
    subject:Optional[str] = None
    body: Optional[str] = None

class Movie(BaseModel):
    movie_name: Optional[str] = None
    description: Optional[str] = None
    year: Optional[int] = None

class Genre(BaseModel):
    genre: Optional[str] = Field(None, description="e.g., comedy, drama, action")   

#### Define Transducible Functions

In [2]:
@transducible(provide_explanation=False)
async def write_an_email(state: GenericInput) -> Email:
    """Write an email about the provided content. Elaborate on that and make up content as needed"""
    return Transduce(state)

## Transducible functions can be introspected to easily get their input , output , description and original function
print(write_an_email.input_model)
print(write_an_email.target_model)
print(write_an_email.description)
print(write_an_email.__original_fn__)


single_mail, explanation = await write_an_email(GenericInput(content="Hi Lisa, I made great progress with the new release of Agentics 2.0"))
print(single_mail.model_dump_json(indent=2))



<class '__main__.GenericInput'>
<class '__main__.Email'>
Write an email about the provided content. Elaborate on that and make up content as needed
<function write_an_email at 0x12f90aa20>
{
  "to": "Lisa",
  "subject": "Exciting Progress Update: Agentics 2.0 New Release",
  "body": "Hi Lisa,\n\nI hope you're having a great week. I wanted to reach out and share some exciting news regarding our recent development cycle. I've made great progress with the new release of Agentics 2.0.\n\nThe core functionalities are now stable, and the performance improvements we discussed are exceeding our initial benchmarks. I’m currently finalizing the documentation and polishing the user interface to ensure a seamless transition for our users. \n\nI’d love to walk you through a demo of the new features once I have the final build ready later this week. Let me know when you might have a few minutes to catch up.\n\nBest regards,\n\n[Your Name]"
}


### Hybrid (Code and llm) functions

A transducible function must follow two rules:

1. It must accept exactly one input parameter : The parameter must be an instance of a Pydantic SOURCE type.
This enforces strong typing and guarantees predictable I/O behavior.

2. It must return one of the following:
- A TARGET Pydantic object: If the function directly returns an instance of the TARGET type, no LLM call is made — the function behaves as a pure Python transformation.
- B. Transduce(obj) :If it returns Transduce(obj) (where obj is a SOURCE-type Pydantic object), this triggers an LLM-based transduction, meaning: the input object is serialized, instructions are applied, the LLM produces an output object of the TARGET type via structured decoding.

This mechanism turns a simple Python function into a fully controllable, typed, LLM-driven transformation.

In [3]:
import re

@transducible(provide_explanation=True)
async def write_an_email_code_only(state: GenericInput) -> Email:  
    match = re.match(r"^(Hi|Dear|Hello|Hey)\s+([^,]+),\s*(.+)$", state.content)
    if match:
        greeting, name, body = match.groups()
        return Email(body= body, to=name, subject="")
    else: return Email()

@transducible(provide_explanation=True)
async def write_an_email_to_lisa(state: GenericInput) -> Email:
    """Write an email about the provided content. Elaborate on that and make up content as needed"""
    # example code to modify states before transduction
    state.content=state.content + " send it to Lisa"
    return Transduce(state)

code_only_mail = await write_an_email_code_only(GenericInput(content=f"Hi Lisa, I have made great progress with agentics"))
hybrid_code_llm_mail, explanation = await write_an_email_to_lisa(GenericInput(content=f"I have made great progress with agentics"))

print(code_only_mail.model_dump_json(indent=2))
print(hybrid_code_llm_mail.model_dump_json(indent=2))
print(explanation)

{
  "to": "Lisa",
  "subject": "",
  "body": "I have made great progress with agentics"
}
{
  "to": "Lisa",
  "subject": "Update on Agentics Progress",
  "body": "Hi Lisa,\n\nI am writing to share some exciting news regarding my recent work. I have made significant progress with agentics and have successfully reached several key milestones in the development process. The systems are becoming increasingly autonomous and efficient, exceeding the initial performance benchmarks.\n\nI wanted to ensure you were kept in the loop on these advancements. Please let me know if you would like to schedule a brief meeting to review the latest updates in more detail.\n\nBest regards,"
}
explanation="The transduction is partially supported but contains significant hallucinated content. The 'to' field ('Lisa') is directly derived from the source instruction 'send it to Lisa'. The 'subject' and 'body' fields are logically linked to the source phrase 'made great progress with agentics', but the specific 

### Using tools

transducible functions can take a list of tools as an argument when they are initialized . 
You can use both CrewAI tools and MCP tools seamlessy. 

In [4]:
from crewai.tools import tool
from ddgs import DDGS
@tool("web_search")
def web_search(query: str) -> str:
    """return spippets of text extracted from duck duck go search for the given
        query :  using DDGS search operators
        
    DDGS search operators Guidelines in the table below:
    Query example	Result
    cats dogs	Results about cats or dogs
    "cats and dogs"	Results for exact term "cats and dogs". If no results are found, related results are shown.
    cats -dogs	Fewer dogs in results
    cats +dogs	More dogs in results
    cats filetype:pdf	PDFs about cats. Supported file types: pdf, doc(x), xls(x), ppt(x), html
    dogs site:example.com	Pages about dogs from example.com
    cats -site:example.com	Pages about cats, excluding example.com
    intitle:dogs	Page title includes the word "dogs"
    inurl:cats	Page url includes the word "cats"""
    return str(DDGS().text(query, max_results=20))


When using tools , you can set reasoning=True to use the planning strategy implemented by crewAI. Setting verbose_agent=True print out agent logs, max_iter is the maximun number of steps (mostly tool calls) allowed before executing a single transduction

In [5]:
class WebSearchResult(BaseModel):
    report_summary:Optional[str]=None
    relevant_sources:Optional[list[str]]=None


@transducible(tools=[web_search], reasoning=True, max_iter=20, provide_explanation=False)
async def answer_question_after_lookup(query: GenericInput) -> WebSearchResult:
    "perform an extensive web search to provide an answer to the input question with supporting evidence. Use your tool to look it up" 
    return Transduce(query)

out , explanation= await answer_question_after_lookup(GenericInput(content="who was the NYC mayor in 2025 and in 1998?"))
print(out.model_dump_json(indent=2))
print(explanation)

{
  "report_summary": "In 1998, the Mayor of New York City was Rudolph W. Giuliani, who held the office from 1994 to 2001. In 2025, the Mayor of New York City was Eric L. Adams, who served from January 1, 2022, through 2025. Following the mayoral election on November 4, 2025, Zohran Mamdani was elected as the next mayor of the city.",
  "relevant_sources": [
    "https://www.nyc.gov/site/dcas/about/green-book-mayors-of-the-city-of-new-york.page",
    "https://en.wikipedia.org/wiki/List_of_mayors_of_New_York_City",
    "https://abcnews.com/Politics/new-york-city-2025-mayoral-election-results-mamdani/story?id=126345335",
    "https://www.dw.com/en/new-york-city-mayoral-election/a-74591495"
  ]
}
None


### Explainability

One of the key strengths of transducible functions is that they don’t just give you an answer —  
they can also give you a **typed explanation** of *why* that answer was produced.

By setting `provide_explanation=True` on a transducible function, Agentics returns:

- the **main output state** (e.g., a classification), and  
- a separate **explanation state** (another Pydantic model) capturing rationale and evidence.

Below is a minimal example using a `Movie → Genre` classifier.

In [6]:
from agentics.core.transducible_functions import Transduce, transducible


@transducible(provide_explanation=False, prompt_template="{movie_name}: {description}")
async def classify_genre(state:Movie)-> Genre:
    """Classify the genre of the source Movie """
    return Transduce(state)

from agentics.core.transducible_functions import _unpack_if_needed

genre, explanation = _unpack_if_needed(await classify_genre(Movie(
    movie_name="The Godfather",
    description="The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.",
    year=1972
)))
print(genre.model_dump_json(indent=2))



{
  "genre": "Crime Drama"
}
